# construct subset of objects

In [1]:
import os
import json

json_path = os.path.join('../../datasets', 'gqa', 'GQA_100', 'fewshot')


In [4]:
# load json

file_path = os.path.join(json_path, 'non_novel', 'non_novel_fewshot.json')
# objects = ('shirt', 'building', 'sign', 'car', 'flower', 'pants', 'helmet', 'pole', 'leaves', 'wall', 'door', 'fence', 'hat', 'shorts')      # fresh
objects = (('plate', 'grass', 'table', 'chair', 'jacket', 'shoe', 'bench'))      # old
new_name = 'nons.json'

info = json.load(open(file_path))


In [5]:

subset = []
for item in info:
    if len(set(item['comb']).difference(objects)) == 0:     # all objects in our desired set
        subset.append(item)

print(len(subset))
print(sorted(list(set([tuple(sorted(item['comb'])) for item in subset]))))

1000
[('bench', 'chair'), ('bench', 'jacket'), ('bench', 'table'), ('chair', 'grass'), ('chair', 'shoe'), ('grass', 'jacket'), ('grass', 'plate'), ('grass', 'shoe'), ('grass', 'table'), ('jacket', 'shoe')]


In [12]:
# save subset

# with open(os.path.join(json_path, new_name), 'w') as f:
#     json.dump(subset, f)

In [7]:
# for sys

file_path = os.path.join(json_path, 'sys', 'sys_fewshot.json')
objects = ('shirt', 'building', 'sign', 'car', 'flower', 'pants', 'helmet', 'pole', 'leaves', 'wall', 'door', 'fence', 'hat', 'shorts')      # fresh
# objects = (('plate', 'grass', 'table', 'chair', 'jacket', 'shoe', 'bench'))      # old
new_name = 'syss.json'

info = json.load(open(file_path))

subset = []
for item in info:
    if len(set(item['comb']).difference(objects)) == 0:     # all objects in our desired set
        subset.append(item)

print(len(subset))
print(sorted(list(set([tuple(sorted(item['comb'])) for item in subset]))))

# # save subset
#
# with open(os.path.join(json_path, new_name), 'w') as f:
#     json.dump(subset, f)

4200
[('building', 'car'), ('building', 'door'), ('building', 'fence'), ('building', 'helmet'), ('building', 'pants'), ('building', 'shirt'), ('building', 'shorts'), ('building', 'wall'), ('car', 'door'), ('car', 'fence'), ('car', 'hat'), ('car', 'pole'), ('car', 'wall'), ('door', 'flower'), ('door', 'hat'), ('door', 'helmet'), ('door', 'pants'), ('door', 'shirt'), ('door', 'shorts'), ('door', 'sign'), ('door', 'wall'), ('fence', 'hat'), ('fence', 'shirt'), ('fence', 'sign'), ('flower', 'hat'), ('flower', 'leaves'), ('flower', 'shorts'), ('flower', 'sign'), ('hat', 'helmet'), ('hat', 'leaves'), ('hat', 'pants'), ('helmet', 'pants'), ('leaves', 'pants'), ('leaves', 'pole'), ('pants', 'pole'), ('pants', 'sign'), ('pole', 'shorts'), ('pole', 'sign'), ('shirt', 'shorts'), ('shirt', 'sign'), ('shorts', 'wall'), ('sign', 'wall')]


# Generate results table

In [2]:
import numpy as np
import os
import pandas as pd

def get_accs(exp_name, mode='sys', project_name='CGQA', name='Top1_Acc_Stream/eval_phase/test_stream'):
    if not os.path.exists(os.path.join('../../avalanche-experiments', project_name, exp_name, f'results-{exp_name}.npy')):
        return np.array([[np.nan]])
    if mode == 'continual':
        results = np.load(os.path.join('../../avalanche-experiments', project_name, exp_name, f'results-{exp_name}.npy'), allow_pickle=True)
    else:
        results = np.load(os.path.join('../../avalanche-experiments', project_name, exp_name, f'results-{exp_name}-{mode}-naive-frz.npy'), allow_pickle=True)

    accs = []

    for k, v in results[-1].items():
        if k.startswith(name):
            accs.append(v)

    return np.array(accs)

# '''Load one exp'''
# exp_name = 'Naive-tsk'
# results = np.load(os.path.join('../../avalanche-experiments', project_name, exp_name, f'results-{exp_name}.npy'), allow_pickle=True)
# last_result = results[-1]
# # print('Average test acc:', get_average_metric(results[-1], 'Top1_Acc_Stream/eval_phase/test_stream'))

In [21]:


modes = ['continual', 'sys', 'pro', 'sub', 'non', 'noc']
# exps = ['naive-cls-lr0_003', 'er-cls-lr0_003', 'gem-cls-lr0_01-p32-m0_3', 'lwf-cls-lr0_005-a1-t1', 'ewc-cls-lr0_005-lambda0_1']
# exps_dis = ['Finetune', 'ER', 'GEM', 'LwF', 'EWC']
exps = ['naive-tsk-lr0_008', 'er-tsk-lr0_0008', 'gem-tsk-lr0_001-p32-m0_3', 'lwf-tsk-lr0_01-a1-t1', 'ewc-tsk-lr0_005-lambda2']
exps_dis = ['Finetune*', 'ER*', 'GEM*', 'LwF*', 'EWC*']

collected_data = []
for exp_idx, exp_name in enumerate(exps):
    for mode in modes:
        accs = get_accs(exp_name, mode, 'CGQA')

        collected_data.append(pd.DataFrame({'Method': exps_dis[exp_idx], 'Phase': mode, 'Accuracy': accs}))

collected_data = pd.concat(collected_data, ignore_index=True)

In [22]:
'''cal mean and ci95, then formulate table str-cell and to latex.'''
# exp = 'Finetune'
# mode = 'continual'
# acc_list = collected_data[(collected_data['Method'] == exp) & (collected_data['Phase'] == mode)]['Accuracy']
data = []
for exp in exps_dis:
    for mode in modes:
        acc_list = collected_data[(collected_data['Method'] == exp) & (collected_data['Phase'] == mode)]['Accuracy']
        mean = acc_list.mean()
        ci95 = 1.96 * (acc_list.std()/np.sqrt(len(acc_list)))
        acc_str = f'{mean*100:.2f} +- {ci95*100:.2f}' if mode != 'continual' else f'{mean*100:.2f}'
        data.append(pd.DataFrame({'Method': exp, 'Phase': mode, 'mean': mean, 'ci95': ci95, 'str': acc_str}, index=[0]))
data = pd.concat(data, ignore_index=True)
# print(data)

# to latex

data = data.pivot(index='Method', values='str', columns='Phase')
data = data[modes]
data = data.reindex(exps_dis)
print(data)
print(data.style.to_latex().replace('+-', '$\\pm$'))

Phase    continual            sys            pro            sub  \
Method                                                            
Finetune     72.46  70.32 +- 0.73  72.62 +- 0.63  66.33 +- 0.69   
ER           76.05  71.37 +- 0.70  72.67 +- 0.69  66.80 +- 0.63   
GEM          21.60  69.44 +- 0.75  73.14 +- 0.67  66.61 +- 0.66   
LwF          73.19  69.61 +- 0.77  74.22 +- 0.63  65.00 +- 0.65   
EWC          71.10  69.40 +- 0.79  72.37 +- 0.65  65.10 +- 0.67   

Phase               non            noc  
Method                                  
Finetune  75.32 +- 0.70  43.26 +- 0.73  
ER        76.28 +- 0.66  45.61 +- 0.77  
GEM       73.31 +- 0.71  45.97 +- 0.74  
LwF       75.25 +- 0.69  42.52 +- 0.72  
EWC       74.99 +- 0.72  42.82 +- 0.77  
\begin{tabular}{lllllll}
Phase & continual & sys & pro & sub & non & noc \\
Method &  &  &  &  &  &  \\
Finetune & 72.46 & 70.32 $\pm$ 0.73 & 72.62 $\pm$ 0.63 & 66.33 $\pm$ 0.69 & 75.32 $\pm$ 0.70 & 43.26 $\pm$ 0.73 \\
ER & 76.05 & 71.37 $\pm$